### classify newsgroup docsby considering each document a simple mean of its individual word embeddings, using one-vs-all SVM

In [1]:
import csv
import os
import re
import sys
import numpy as np
import tensorflow as tf

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation, Lambda
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing, svm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import ParameterGrid
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.pipeline import Pipeline

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
from keras import backend as K

In [3]:
SEED=np.random.randint(1,1000)
SEED

186

In [4]:
np.random.seed(seed=SEED)

In [5]:
texts,labels_index,labels_0 = files_helper.read_20_newgroup_files()       

In [6]:
preprocessed_texts = texts

In [7]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)
BATCH_SIZE=32
NUM_EPOCHS=10
TOKENIZER_FILTERS='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' # default keras filters

In [8]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters=TOKENIZER_FILTERS)
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [9]:
[inverse_word_index[idx] for idx in sequences[0]]

['archive',
 'name',
 'atheism',
 'resources',
 'alt',
 'atheism',
 'archive',
 'name',
 'resources',
 'last',
 'modified',
 '11',
 'december',
 '1992',
 'version',
 '1',
 '0',
 'atheist',
 'resources',
 'addresses',
 'of',
 'atheist',
 'organizations',
 'usa',
 'freedom',
 'from',
 'religion',
 'foundation',
 'darwin',
 'fish',
 'bumper',
 'stickers',
 'and',
 'assorted',
 'other',
 'atheist',
 'are',
 'available',
 'from',
 'the',
 'freedom',
 'from',
 'religion',
 'foundation',
 'in',
 'the',
 'us',
 'write',
 'to',
 'p',
 'o',
 'box',
 '750',
 'madison',
 'wi',
 'telephone',
 '608',
 '256',
 '8900',
 'evolution',
 'designs',
 'evolution',
 'designs',
 'sell',
 'the',
 'darwin',
 'fish',
 "it's",
 'a',
 'fish',
 'symbol',
 'like',
 'the',
 'ones',
 'christians',
 'stick',
 'on',
 'their',
 'cars',
 'but',
 'with',
 'feet',
 'and',
 'the',
 'word',
 'darwin',
 'written',
 'inside',
 'the',
 'deluxe',
 '3d',
 'plastic',
 'fish',
 'is',
 '4',
 '95',
 'in',
 'the',
 'us',
 'write',
 'to

In [10]:
# word => embedding
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [11]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

labels_1 = to_categorical(np.asarray(labels_0))

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_2 = labels_1[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_2[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_2[-num_validation_samples:]

data.shape,labels_2.shape

((19997, 1000), (19997, 20))

In [12]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
        
embedding_matrix.shape        

(174075, 100)

In [13]:
num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

# no need for extra layers if we aren't training the pooling layers
model.add(GlobalAvgPool1D())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_labels, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [14]:
%%time

model.fit(X_train,Y_train, validation_data=(X_val, Y_val),
         epochs=NUM_EPOCHS, batch_size=BATCH_SIZE)

Train on 15998 samples, validate on 3999 samples
Epoch 1/10
15998/15998 [==============================] - 2s - loss: 2.9207 - acc: 0.0818 - val_loss: 2.7192 - val_acc: 0.2068
Epoch 2/10
15998/15998 [==============================] - 1s - loss: 2.5625 - acc: 0.1663 - val_loss: 2.2885 - val_acc: 0.2471
Epoch 3/10
15998/15998 [==============================] - 2s - loss: 2.3254 - acc: 0.2115 - val_loss: 2.1169 - val_acc: 0.2911
Epoch 4/10
15998/15998 [==============================] - 2s - loss: 2.1924 - acc: 0.2533 - val_loss: 1.9990 - val_acc: 0.3428
Epoch 5/10
15998/15998 [==============================] - 1s - loss: 2.1097 - acc: 0.2768 - val_loss: 1.9154 - val_acc: 0.3598
Epoch 6/10
15998/15998 [==============================] - 1s - loss: 2.0366 - acc: 0.2942 - val_loss: 1.8535 - val_acc: 0.3831
Epoch 7/10
15998/15998 [==============================] - 2s - loss: 1.9874 - acc: 0.3160 - val_loss: 1.8067 - val_acc: 0.4096
Epoch 8/10
15998/15998 [==============================] - 2s -

In [15]:
get_pooling_layer_output = K.function([model.layers[0].input, K.learning_phase()],
                                  [model.layers[1].output])

In [16]:
new_X_train = []
for raw_doc_features in X_train:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1)])
    new_X_train.append(layer_output[0].ravel())
new_X_train = np.vstack(new_X_train)

new_X_train.shape

(15998, 100)

In [17]:
new_X_val = []
for raw_doc_features in X_val:
    layer_output = get_pooling_layer_output([raw_doc_features.reshape(1,-1)])
    new_X_val.append(layer_output[0].ravel())
new_X_val = np.vstack([new_X_val])

new_X_val.shape

(3999, 100)

In [18]:
clf = OneVsRestClassifier(Pipeline([
    ('clf', svm.LinearSVC()),
]),n_jobs=-1)
parameters = [
    { 
          "estimator__clf__penalty": ["l1"],
          "estimator__clf__dual":[False],
          "estimator__clf__multi_class":["ovr"],
          "estimator__clf__max_iter":[1000,2000],
          "estimator__clf__tol": [0.001],

    },
    { 
          "estimator__clf__penalty": ["l2"],
          "estimator__clf__dual":[False,True],
          "estimator__clf__multi_class":["ovr"],
          "estimator__clf__max_iter":[1000,2000],
          "estimator__clf__tol": [0.001],

    },    
]

In [19]:
best_score = float("-inf")

for g in ParameterGrid(parameters):
    clf.set_params(**g)
    %time clf.fit(new_X_train,Y_train)
    
    Y_pred_train = clf.predict(new_X_train)    
    Y_pred_val = clf.predict(new_X_val)
    
    train_score = accuracy_score(Y_train,Y_pred_train)
    val_score = accuracy_score(Y_val,Y_pred_val)
    
    current_score = val_score
    
    print("train accuracy: {}".format(train_score))
    print("val accuracy: {}".format(val_score))
    print("grid: {}".format(g))
    print("")
    
    if current_score > best_score:
        best_score = current_score
        best_grid = g

CPU times: user 704 ms, sys: 276 ms, total: 980 ms
Wall time: 46.9 s
train accuracy: 0.23371671458932367
val accuracy: 0.2213053263315829
grid: {'estimator__clf__max_iter': 1000, 'estimator__clf__dual': False, 'estimator__clf__multi_class': 'ovr', 'estimator__clf__penalty': 'l1', 'estimator__clf__tol': 0.001}

CPU times: user 932 ms, sys: 836 ms, total: 1.77 s
Wall time: 1min 31s
train accuracy: 0.23709213651706462
val accuracy: 0.2220555138784696
grid: {'estimator__clf__max_iter': 2000, 'estimator__clf__dual': False, 'estimator__clf__multi_class': 'ovr', 'estimator__clf__penalty': 'l1', 'estimator__clf__tol': 0.001}

CPU times: user 524 ms, sys: 516 ms, total: 1.04 s
Wall time: 2.95 s
train accuracy: 0.15076884610576322
val accuracy: 0.14428607151787948
grid: {'estimator__clf__max_iter': 1000, 'estimator__clf__dual': False, 'estimator__clf__multi_class': 'ovr', 'estimator__clf__penalty': 'l2', 'estimator__clf__tol': 0.001}

CPU times: user 496 ms, sys: 556 ms, total: 1.05 s
Wall time: